In [1]:
import pyforest 
import requests
from bs4 import BeautifulSoup

In [2]:
%%bash 
cat 'project notes'

1) Give user the option of choosing the country and job type. Will have to string format the link 
2) Some of the jobs have incomplete links. Will have to concat to make the link complete. Use regex and for loop to check if starts with https.
3) Will have to set a time between scrape for each page and then for scraping of each link. Have to set cap at number of total jobs as well. 
4) Ask the user for the list of skills he want a summary. That way we can expand it to incorporate any job title and skill set. 

In [4]:
import contextlib
import time

@contextlib.contextmanager
def timer():
    '''Calculate time it takes for process to complete
    
    Args:
      None
      
    Yields:
       float : the time in minutes for process to run
    '''
    start_time = time.time()
    
    yield
    
    end_time = time.time()
    
    final_time = (end_time - start_time) / 60
    
    print("The time it took to scrape and compile the results was : {:.2f} minutes".format(final_time))
    

In [64]:
from random import randint

print('Hello and welcome to the skills summariser. Let\'s get going')
print('')

job_title = input('Enter the job title you want to search for: ')
country = input('Which country would you like to search in (no abbrevations please): ')
skills = input('Enter the skills you want a summary for and make sure you seperate each skill by a space: ')
n_jobs = input('How many jobs do you want to look through. The more the jobs the longer the script might take: ')
#initial cleaning 

job_title = job_title.title()
country = country.title()
skills = skills.lower().split()

#'https://www.indeed.com/jobs?q=business+analyst&l=United+States'
primary_link = 'https://www.indeed.com/jobs?q={0}&l={1}'
further_pages_link = 'https://www.indeed.com/jobs?q={0}&l={1}&start={3}'


with timer():
    job_df = pd.DataFrame(columns = ['Job Title', 'Organization', 'Job Link'])
    num = 1
    start_range = list(range(10,20000,10))
    skills_dict = {}
    titles = []
    orgs = []
    clean_links = []

    
    
#     while job_df.shape[0] <= int(n_jobs):
#         if num == 1:
    link = primary_link.format(job_title,country)
    job_links_page = requests.get(link).text
    first_page_parser = BeautifulSoup(job_links_page,'html.parser')
    scraped_first_page = first_page_parser.body.find_all('div', attrs = {'data-tn-component':'organicJob'})
            
    for job in scraped_first_page:
        title = job.a.text.strip()
        titles.append(title)
    
        org = job.div.span.text.strip()
        orgs.append(org)
    
        link = job.a.get('href')
    
        if link.startswith('https'):
                clean_links.append(link)
        else:
            join_link = "https://www.indeed.com"+link
            clean_links.append(join_link)
            
    job_df['Job Title'] = titles
    job_df['Organization'] = orgs
    job_df['Job Link'] = clean_links
            
    for l in clean_links:
        try:
            jd_request = requests.get(l)
            if jd_request.status_code == 200:
                jd_cont = jd_request.text
                job_page = BeautifulSoup(jd_cont,'html.parser')
                description = job_page.find('div', class_ = 'jobsearch-jobDescriptionText').text
                description = description.lower()
        
                for skill in skills:
                
                    if skill in description:
                        if skill in skills_dict:
                            skills_dict[skill] +=1
                        else:
                            skills_dict[skill] = 1
                        
                
        except:
            continue

#the code for scraping page 2 and forward starts from here

        


Hello and welcome to the skills summariser. Let's get going

Enter the job title you want to search for: data scientist
Which country would you like to search in (no abbrevations please): united states
Enter the skills you want a summary for and make sure you seperate each skill by a space: python sql 
How many jobs do you want to look through. The more the jobs the longer the script might take: 12


<IPython.core.display.Javascript object>

The time it took to scrape and compile the results was : 0.10 minutes


In [65]:
skills_dict

{'python': 13, 'sql': 8}

In [61]:
job_df

,Job Title,Organization,Job Link
0,Entry Level Data Analyst,Boiling Point,https://www.indeed.com/company/Boiling-Point/j...
1,Clinical Data Analyst,"WithHealth, Inc.","https://www.indeed.com/company/WithHealth,-Inc..."
2,DATA ANALYST,"Clare Foundation, Inc.",https://www.indeed.com/rc/clk?jk=f32a20440a730...
3,Fall Internship Program – Business Analyst Int...,Bigge Crane and Rigging Co.,https://www.indeed.com/company/Bigge-Crane-and...
4,Data Analyst,CallSource,https://www.indeed.com/company/CallSource/jobs...
5,Data Analyst with Tableau,Pyramid,https://www.indeed.com/company/Pyramid/jobs/Da...
6,Data Analyst,Amazon.com Services LLC,https://www.indeed.com/rc/clk?jk=b75f4273a9510...
7,Entry Level Business Analyst,Otcmarketpro,https://www.indeed.com/company/Otcmarketpro/jo...
8,Data Analyst - Temporary,"Refuse Specialists, LLC",https://www.indeed.com/company/Refuse-Speciali...
